# SCOPE Router - Complete Guide

This notebook provides a complete guide to running SCOPE router, including:
1. **Default Experiment** - Using built-in dataset and model pool
2. **Custom Model Pool** - Using your own set of models
3. **Custom Query Set** - Routing your own questions
4. **Advanced Configuration** - Customizing pricing, parameters, etc.

---
## 0. Setup Environment

In [ ]:
# Check GPU
!nvidia-smi

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Clone or update repository
import os

REPO_PATH = "/content/drive/MyDrive/SCOPE"

if os.path.exists(REPO_PATH):
    print("Repository exists, updating...")
    %cd {REPO_PATH}
    !git pull origin main
else:
    print("Cloning repository...")
    %cd /content/drive/MyDrive
    !git clone https://github.com/Sullivan07043/SCOPE.git

In [ ]:
# Install dependencies
!pip install vllm>=0.4.0 transformers>=4.40.0 datasets torch numpy tqdm requests huggingface_hub -q

In [ ]:
# Set environment variables for caching
import os

CACHE_DIR = "/content/drive/MyDrive/hf_cache"
os.makedirs(CACHE_DIR, exist_ok=True)

os.environ['HF_HOME'] = CACHE_DIR
os.environ['TRANSFORMERS_CACHE'] = CACHE_DIR
os.environ['HF_DATASETS_CACHE'] = CACHE_DIR
os.environ['HUGGINGFACE_HUB_CACHE'] = CACHE_DIR
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

print(f"Cache directory: {CACHE_DIR}")

In [ ]:
# Define paths
REPO_PATH = "/content/drive/MyDrive/SCOPE"
SCRIPTS_PATH = f"{REPO_PATH}/scripts"
RESULTS_PATH = f"{REPO_PATH}/results"
EXAMPLES_PATH = f"{REPO_PATH}/examples"

os.makedirs(RESULTS_PATH, exist_ok=True)
os.chdir(SCRIPTS_PATH)
print(f"Working directory: {os.getcwd()}")

---
## 1. Default Experiment

Run SCOPE with default settings:
- **Dataset**: Cooolder/SCOPE-60K-final (In-Distribution, 252 test questions)
- **Model Pool**: DEFAULT_POOL (7 models)
- **SCOPE Model**: Cooolder/SCOPE

In [ ]:
# Step 1: Run SCOPE Inference
# This generates predictions for each query-model pair

!python scope_inference.py \
    --dataset id \
    --output "{RESULTS_PATH}/selection.json" \
    --similarity_output "{RESULTS_PATH}/similarity.json" \
    --cache_dir "{CACHE_DIR}"

In [ ]:
# Step 2: Run Two-Stage Routing with budget constraint

!python two_stage_routing.py \
    --selection "{RESULTS_PATH}/selection.json" \
    --similarity "{RESULTS_PATH}/similarity.json" \
    --budget 5.0 \
    --output "{RESULTS_PATH}/routing_results.json" \
    --dataset id

In [ ]:
# View routing results
import json

with open(f"{RESULTS_PATH}/routing_results.json", 'r') as f:
    results = json.load(f)

print("=" * 50)
print("Routing Results")
print("=" * 50)
print(f"Optimal Alpha: {results.get('optimal_alpha', 'N/A')}")
print(f"Accuracy: {results['result']['accuracy']*100:.2f}%")
print(f"Total Cost: ${results['result']['total_cost']:.4f}")
print(f"\nModel Distribution:")
for model, count in results['result']['model_distribution'].items():
    print(f"  {model}: {count}")

---
## 2. Using Different Model Pools

Available pools:
- `default` (7 models) - Balanced across cost tiers
- `full` (13 models) - All in-distribution models
- `reasoning` (5 models) - Optimized for complex reasoning
- `high_budget` (4 models) - Premium models
- `low_budget` (5 models) - Cost-efficient models

In [ ]:
# Example: Use FULL model pool (13 models)

!python scope_inference.py \
    --dataset id \
    --pool_name full \
    --output "{RESULTS_PATH}/selection_full.json" \
    --similarity_output "{RESULTS_PATH}/similarity.json" \
    --cache_dir "{CACHE_DIR}"

In [ ]:
# Routing with full pool

!python two_stage_routing.py \
    --selection "{RESULTS_PATH}/selection_full.json" \
    --similarity "{RESULTS_PATH}/similarity.json" \
    --budget 10.0 \
    --output "{RESULTS_PATH}/routing_results_full.json"

In [ ]:
# You can also filter models during routing
# Example: Use selection from full pool but route with only default pool

!python two_stage_routing.py \
    --selection "{RESULTS_PATH}/selection_full.json" \
    --similarity "{RESULTS_PATH}/similarity.json" \
    --pool_name default \
    --budget 5.0 \
    --output "{RESULTS_PATH}/routing_results_filtered.json"

---
## 3. Custom Query Set

Route your own questions instead of using the default test set.

In [ ]:
# Create a custom query file

custom_queries = [
    {
        "id": "my_q001",
        "prompt": "What is the capital of France?\n\nA. London\nB. Paris\nC. Berlin\nD. Madrid",
        "gt": "B",
        "category": "geography"
    },
    {
        "id": "my_q002",
        "prompt": "Solve for x: 2x + 5 = 13\n\nA. x = 3\nB. x = 4\nC. x = 5\nD. x = 6",
        "gt": "B",
        "category": "math"
    },
    {
        "id": "my_q003",
        "prompt": "Which programming language is known for data science?\n\nA. Java\nB. Python\nC. C++\nD. Ruby",
        "gt": "B",
        "category": "programming"
    },
    {
        "id": "my_q004",
        "prompt": "What is the chemical symbol for gold?\n\nA. Ag\nB. Fe\nC. Au\nD. Cu",
        "gt": "C",
        "category": "chemistry"
    },
    {
        "id": "my_q005",
        "prompt": "What is the derivative of x^3?\n\nA. x^2\nB. 3x^2\nC. 3x\nD. x^4",
        "gt": "B",
        "category": "calculus"
    }
]

import json
with open(f"{RESULTS_PATH}/my_queries.json", 'w') as f:
    json.dump(custom_queries, f, indent=2)

print(f"Created custom query file with {len(custom_queries)} questions")

In [ ]:
# Run SCOPE inference on custom queries

!python scope_inference.py \
    --query_file "{RESULTS_PATH}/my_queries.json" \
    --output "{RESULTS_PATH}/selection_custom.json" \
    --similarity_output "{RESULTS_PATH}/similarity_custom.json" \
    --cache_dir "{CACHE_DIR}"

In [ ]:
# Run routing on custom queries

!python two_stage_routing.py \
    --selection "{RESULTS_PATH}/selection_custom.json" \
    --similarity "{RESULTS_PATH}/similarity_custom.json" \
    --budget 1.0 \
    --output "{RESULTS_PATH}/routing_custom.json"

In [ ]:
# View custom routing results
with open(f"{RESULTS_PATH}/routing_custom.json", 'r') as f:
    results = json.load(f)

print("Custom Query Routing Results")
print("=" * 50)
for decision in results['result']['routing_decisions']:
    print(f"Question: {decision['question_id']}")
    print(f"  Selected Model: {decision['selected_model']}")
    print(f"  Cost: ${decision['cost']:.4f}")
    print()

---
## 4. Custom Model Pool (Advanced)

Use your own set of models. This requires:
1. Creating a model pool file
2. Setting up pricing
3. Running anchor inference (requires OpenRouter API key)

**Note**: This section requires an OpenRouter API key and will incur API costs.

In [ ]:
# Step 1: Create custom model pool file

custom_models = """
# My custom model pool
google/gemma-3-27b-it
meta-llama/llama-3.3-70b-instruct
qwen/qwen3-14b
"""

with open(f"{RESULTS_PATH}/my_models.txt", 'w') as f:
    f.write(custom_models.strip())

print("Created custom model pool file")
!cat "{RESULTS_PATH}/my_models.txt"

In [ ]:
# Step 2: Update pricing for custom models
# Edit config/pricing.json to add your models' pricing

import json

pricing_file = f"{REPO_PATH}/config/pricing.json"
with open(pricing_file, 'r') as f:
    pricing = json.load(f)

print("Current pricing configuration:")
for model, price in pricing.items():
    if not model.startswith('_'):
        print(f"  {model}: input=${price['input']}, output=${price['output']}")

In [ ]:
# Step 3: Run anchor inference (REQUIRES OpenRouter API KEY)
# This step calls the OpenRouter API and will incur costs

# Set your OpenRouter API key
import os
os.environ['OPENROUTER_API_KEY'] = 'your_api_key_here'  # <-- REPLACE THIS

# Uncomment below to run anchor inference
# WARNING: This will make API calls and incur costs!

# !python inference_anchor.py \
#     --model_pool "{RESULTS_PATH}/my_models.txt" \
#     --output "{RESULTS_PATH}/my_anchor_results/" \
#     --dataset id \
#     --limit 10  # Start with small limit for testing

In [ ]:
# Step 4: Run SCOPE inference with custom model pool
# Only run this after anchor inference is complete

# !python scope_inference.py \
#     --dataset id \
#     --model_pool "{RESULTS_PATH}/my_models.txt" \
#     --anchor_dir "{RESULTS_PATH}/my_anchor_results/" \
#     --output "{RESULTS_PATH}/selection_custom_pool.json" \
#     --similarity_output "{RESULTS_PATH}/similarity.json" \
#     --cache_dir "{CACHE_DIR}"

In [ ]:
# Step 5: Run routing with custom pricing file

# !python two_stage_routing.py \
#     --selection "{RESULTS_PATH}/selection_custom_pool.json" \
#     --similarity "{RESULTS_PATH}/similarity.json" \
#     --pricing_file "{REPO_PATH}/config/pricing.json" \
#     --budget 5.0 \
#     --output "{RESULTS_PATH}/routing_custom_pool.json"

---
## 5. Advanced Configuration

In [ ]:
# Routing with custom alpha (instead of budget)
# alpha=0 -> minimize cost
# alpha=1 -> maximize accuracy

!python two_stage_routing.py \
    --selection "{RESULTS_PATH}/selection.json" \
    --similarity "{RESULTS_PATH}/similarity.json" \
    --alpha 0.7 \
    --output "{RESULTS_PATH}/routing_alpha07.json"

In [ ]:
# Routing with custom algorithm parameters

!python two_stage_routing.py \
    --selection "{RESULTS_PATH}/selection.json" \
    --similarity "{RESULTS_PATH}/similarity.json" \
    --budget 5.0 \
    --top_k 10 \
    --similarity_power 3.0 \
    --cost_sensitivity 1.5 \
    --stage1_weight 0.4 \
    --output "{RESULTS_PATH}/routing_custom_params.json"

In [ ]:
# Generate Pareto curve (scan all alpha values)
# This shows the trade-off between cost and accuracy

!python two_stage_routing.py \
    --selection "{RESULTS_PATH}/selection.json" \
    --similarity "{RESULTS_PATH}/similarity.json" \
    --output "{RESULTS_PATH}/routing_pareto.json"

In [ ]:
# SCOPE inference with custom parameters

!python scope_inference.py \
    --dataset id \
    --output "{RESULTS_PATH}/selection_custom_scope.json" \
    --similarity_output "{RESULTS_PATH}/similarity.json" \
    --num_anchor_examples 7 \
    --top_k_similarity 15 \
    --max_new_tokens 2048 \
    --temperature 0.3 \
    --cache_dir "{CACHE_DIR}" \
    --limit 50  # Limit for testing

---
## 6. Using OOD (Out-of-Distribution) Dataset

In [ ]:
# Run with OOD dataset (5 unseen models)

!python scope_inference.py \
    --dataset ood \
    --output "{RESULTS_PATH}/selection_ood.json" \
    --similarity_output "{RESULTS_PATH}/similarity_ood.json" \
    --cache_dir "{CACHE_DIR}"

In [ ]:
# Routing for OOD dataset

!python two_stage_routing.py \
    --selection "{RESULTS_PATH}/selection_ood.json" \
    --similarity "{RESULTS_PATH}/similarity_ood.json" \
    --dataset ood \
    --budget 5.0 \
    --output "{RESULTS_PATH}/routing_ood.json"

---
## 7. Cleanup

In [ ]:
# List all result files
!ls -la "{RESULTS_PATH}"

In [ ]:
# Optional: Clean up result files (uncomment to run)

# !rm -rf "{RESULTS_PATH}"/*
# print("Results cleaned up")